In [52]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())


In [44]:
from app.engine import Survey, QuestionPro

In [45]:
questionpro = QuestionPro(
    survey_id='12032803', 
    api_key='61d15ae9-8995-4f6b-88ac-3a66e752a3cc'
)
await questionpro.fetch_data()
data = questionpro.data
survey = Survey(data=data)
survey.initialize()


In [67]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.graph.message import add_messages
from langgraph.types import Command
from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent


from typing_extensions import TypedDict, Literal, Annotated, Sequence


In [69]:
llm = ChatOpenAI(model='gpt-3.5-turbo', streaming=True)

search_tool = TavilySearchResults(max_results=1)

@tool
def crosstab(base: str, target: str):
    """Use to see relationship between two varibles by looking for crosstable of them."""
    ctab = survey.crosstab(base=base, target=target)
    return ctab.df.to_string()

@tool
def get_question_info(question_code):
    """Use to get question info by question code"""
    question = survey[question_code]
    answers = ""
    for answer in question.answers:
        answers += f"        --> {answer.scale} - {answer.text} - num_respondent: {len(answer.respondents)}\n"
    describe = f"""
Question(code={question.code}, text={question.text}, type={question.type}, num_answers={len(question.answers)}, num_respondent={len(question.respondents)})
-> answers:
{answers}
"""
    return describe

In [95]:
members = ['crosstaber', 'question_getter']
options = members + ['FINISH']

system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    f" following workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)

In [97]:
class Router(TypedDict):
    """Worker to route to next. If no workers needed, route to FINISH."""
    next: Literal['crosstaber', 'question_getter', 'FINISH']
def supervisor_node(state=MessagesState) -> Command[Literal['crosstaber', 'question_getter', '__end__']]:
    messages = [
        SystemMessage(content=system_prompt)
    ] + state['messages']
    response = llm.with_structured_output(Router).invoke(messages)
    goto = END if response['next'] == 'FINISH' else response['next']
    return Command(goto=goto)

In [96]:
ctab_agent = create_react_agent(llm, tools=[crosstab], state_modifier="You are an analyzer who professional in crosstab data based on two question code provided.")

def ctab_node